In [1]:
from pyuvm import *
from tb_pkg import *
import random
import time

class alu_test(uvm_test):
    @staticmethod
    def alu_op(A, B, op, is_dut = False):
        """TLM model of the TinyALU"""
        assert isinstance(op, Ops), "The tinyalu op must be of type ops"
        if op == Ops.NOP:
            result =  A
        elif op == Ops.ADD:
            result = A + B
        elif op == Ops.AND:
            result = A & B
        elif op == Ops.XOR:
            result = A ^ B
        elif op == Ops.MUL:
            result = A * B
        if is_dut:
            time.sleep(0.1)
            if is_dut and (random.randint(0,3) == 0):
                result = result + 1
        return result

    def run_phase(self):
        self.raise_objection()
        logging.basicConfig(level=logging.INFO)
        cvg = set() #functional coverage
        for _ in range(5):
            A = random.randrange(256)
            B = random.randrange(256)
            op = random.choice(list(Ops))
            cvg.add(op)
            predicted_result = self.alu_op(A, B, op)
            actual_result = self.alu_op(A, B, op, is_dut=True)
            if predicted_result == actual_result:
                self.logger.info( f"PASSED: {A:02x} {op.name} {B:02x} = {actual_result:04x}")
            else:
                self.logger.error(f"FAILED: {A:02x} {op.name} {B:02x} = {actual_result:04x} expected {predicted_result:04x}")
        if len(set(Ops) - cvg) > 0:
            self.logger.error(f"Functional coverage error. Missed: {set(Ops)-cvg}")
        self.drop_objection()
uvm_root().run_test("alu_test")

ERROR: <ipython-input-1-e087adc2ab9d>(41)[uvm_test_top]: FAILED: b9 MUL 27 = 1c30 expected 1c2f
INFO: <ipython-input-1-e087adc2ab9d>(39)[uvm_test_top]: PASSED: b1 MUL 64 = 4524
INFO: <ipython-input-1-e087adc2ab9d>(39)[uvm_test_top]: PASSED: cb ADD 9b = 0166
ERROR: <ipython-input-1-e087adc2ab9d>(41)[uvm_test_top]: FAILED: 54 NOP 4b = 0055 expected 0054
ERROR: <ipython-input-1-e087adc2ab9d>(41)[uvm_test_top]: FAILED: 44 MUL dd = 3ab5 expected 3ab4
ERROR: <ipython-input-1-e087adc2ab9d>(43)[uvm_test_top]: Functional coverage error. Missed: {<Ops.AND: 2>, <Ops.XOR: 3>}
